In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("./datasets/tutorial_sleep_training_data.csv.gz")
df = df.dropna()

# Visual Intuition:

<img src="images/sliding_win.jpg" width="400" height="200">

In [3]:
df1 = df[df["pid"] == 10]
df1

,time,act,sleep_phase,hr,pid
289664,29,38.0,0.0,71.0,10
289694,59,0.0,0.0,69.0,10
289724,89,23.0,0.0,73.0,10
289754,119,17.0,0.0,59.0,10
289784,149,8.0,0.0,71.0,10
...,...,...,...,...,...
323654,34019,64.0,0.0,100.0,10
323684,34049,42.0,0.0,31.0,10
323714,34079,45.0,0.0,34.0,10
323744,34109,105.0,0.0,33.0,10


In [4]:
df1["act"].mean()

8.06414762741652

In [5]:
df1[0:10]["act"].mean(), df1[1:11]["act"].mean(), df1[2:12]["act"].mean()

(10.6, 8.8, 13.2)

In [6]:
# Centered window of size 11 (5 + 1 + 5)
i = 5
winsize = 10
df_slice = df1[i-(winsize//2):i+(winsize//2)+1]

print("Shape: ", df_slice.shape)
df_slice

Shape:  (11, 5)


,time,act,sleep_phase,hr,pid
289664,29,38.0,0.0,71.0,10
289694,59,0.0,0.0,69.0,10
289724,89,23.0,0.0,73.0,10
289754,119,17.0,0.0,59.0,10
289784,149,8.0,0.0,71.0,10
289814,179,12.0,0.0,61.0,10
289844,209,0.0,0.0,73.0,10
289874,239,0.0,0.0,73.0,10
289904,269,0.0,0.0,68.0,10
289934,299,8.0,0.0,80.0,10


In [18]:
# %%timeit

n = df1.shape[0]
winsize = 11

result_loop = []
for i in range(0, n):
    r = df1[i-(winsize//2):i+(winsize//2)+1]["act"].mean()
    result_loop.append(r)
    
result_loop = pd.Series(result_loop)


31.3 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
result_loop[:20]

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5     11.454545
6     12.000000
7     12.181818
8     12.818182
9     11.727273
10    11.000000
11    11.363636
12    13.545455
13    25.909091
14    39.181818
15    45.454545
16    52.818182
17    67.727273
18    72.363636
19    83.727273
dtype: float64

# Pandas windowing
Alternatively, you could use the native version from pandas:

https://pandas.pydata.org/docs/user_guide/window.html

It also has interesting properties to deal with time (e.g., rolling windows per hour)

In [17]:
# %%timeit 

result_pdw = df1["act"].rolling(window=11, center=True, min_periods=11).mean()
result_pdw[:20]

65.2 µs ± 698 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
result_pdw = df1["act"].rolling(window=11, center=False, min_periods=11).mean()
result_pdw[:20]

289664          NaN
289694          NaN
289724          NaN
289754          NaN
289784          NaN
289814          NaN
289844          NaN
289874          NaN
289904          NaN
289934          NaN
289964    11.454545
289994    12.000000
290024    12.181818
290054    12.818182
290084    11.727273
290114    11.000000
290144    11.363636
290174    13.545455
290204    25.909091
290234    39.181818
Name: act, dtype: float64

In [22]:
result_pdw = df1["act"].rolling(window=11, center=False, min_periods=1).mean()
result_pdw[:20]

289664    38.000000
289694    19.000000
289724    20.333333
289754    19.500000
289784    17.200000
289814    16.333333
289844    14.000000
289874    12.250000
289904    10.888889
289934    10.600000
289964    11.454545
289994    12.000000
290024    12.181818
290054    12.818182
290084    11.727273
290114    11.000000
290144    11.363636
290174    13.545455
290204    25.909091
290234    39.181818
Name: act, dtype: float64

## A Few Question:

- How about the last 20 numbers? I.e.,: result_pdw[-20:] Vs result_loop[-20:]
- How about the execution time? (use %%timeit to check it)


In [24]:
Centered = True
Wsize = 11

df1["act"].rolling(window=Wsize, center=Centered, min_periods=1).agg(["mean", "median", "std", "var", "skew", "kurt"])

,mean,median,std,var,skew,kurt,mysum
289664,16.333333,14.5,13.185851,173.866667,0.713228,0.666706,98.0
289694,14.000000,12.0,13.527749,183.000000,0.835845,0.404760,98.0
289724,12.250000,10.0,13.466891,181.357143,1.000748,0.535173,98.0
289754,10.888889,8.0,13.242398,175.361111,1.158859,0.824273,98.0
289784,10.600000,8.0,12.518431,156.711111,1.270328,1.344534,106.0
...,...,...,...,...,...,...,...
323654,61.300000,54.0,43.415435,1884.900000,0.938059,1.207229,613.0
323684,62.111111,54.0,45.968588,2113.111111,0.847039,0.740940,559.0
323714,68.625000,59.0,44.480935,1978.553571,0.823750,1.031684,549.0
323744,56.571429,54.0,30.859127,952.285714,-0.009530,0.702244,396.0


In [25]:
def mymean(x):
    return np.mean(x)

df1["act"].rolling(window=Wsize, center=Centered, min_periods=1).agg(["mean", "median", "std", "var", "skew", "kurt", mymean])

,mean,median,std,var,skew,kurt,mymean
289664,16.333333,14.5,13.185851,173.866667,0.713228,0.666706,16.333333
289694,14.000000,12.0,13.527749,183.000000,0.835845,0.404760,14.000000
289724,12.250000,10.0,13.466891,181.357143,1.000748,0.535173,12.250000
289754,10.888889,8.0,13.242398,175.361111,1.158859,0.824273,10.888889
289784,10.600000,8.0,12.518431,156.711111,1.270328,1.344534,10.600000
...,...,...,...,...,...,...,...
323654,61.300000,54.0,43.415435,1884.900000,0.938059,1.207229,61.300000
323684,62.111111,54.0,45.968588,2113.111111,0.847039,0.740940,62.111111
323714,68.625000,59.0,44.480935,1978.553571,0.823750,1.031684,68.625000
323744,56.571429,54.0,30.859127,952.285714,-0.009530,0.702244,56.571429


## How to apply rolling windows to the whole dataset?

In [26]:
df_indexed = df.set_index(["pid", "time"])
df_indexed

act  sleep_phase    hr
pid time                          
0   29      2.0          0.0  71.0
    59      0.0          0.0  76.0
    89      1.0          0.0  78.0
    119     2.0          0.0  73.0
    149    87.0          0.0  80.0
...         ...          ...   ...
99  27569   9.0          0.0  67.0
    27599  29.0          0.0  65.0
    27629  53.0          0.0  64.0
    27659  74.0          0.0  67.0
    27689  41.0          0.0  69.0

[102859 rows x 3 columns]

In [27]:
X_grped = df_indexed["act"].rolling(window=Wsize, center=Centered, min_periods=1).agg(["mean", "median", "std", "var", "skew", "kurt", "max", "min", "count", "sum"])
X_grped

mean  median        std          var      skew      kurt  \
pid time                                                                   
0   29     15.333333     1.5  35.120744  1233.466667  2.445916  5.986385   
    59     13.285714     1.0  32.515198  1057.238095  2.642249  6.985881   
    89     11.625000     1.0  30.467489   928.267857  2.824207  7.982049   
    119    10.333333     1.0  28.761954   827.250000  2.995208  8.978607   
    149     9.300000     0.5  27.313204   746.011111  3.157003  9.975393   
...              ...     ...        ...          ...       ...       ...   
99  27569  32.600000    35.0  28.336078   802.933333  0.034822 -1.788971   
    27599  36.000000    41.0  27.807373   773.250000 -0.217459 -1.604856   
    27629  33.125000    35.0  28.261218   798.696429  0.060339 -1.596417   
    27659  29.428571    29.0  28.359931   804.285714  0.446974 -1.109392   
    27689  34.333333    35.0  27.623661   763.066667  0.175520 -0.958776   

            max  min  count    sum  
pid time                            
0   29     87.0  0.0    6.0   92.0  
    59     87.0  0.0    7.0   93.0  
    89     87.0  0.0    8.0   93.0  
    119    87.0  0.0    9.0   93.0  
    149    87.0  0.0   10.0   93.0  
...         ...  ...    ...    ...  
99  27569  74.0  0.0   10.0  326.0  
    27599  74.0  0.0    9.0  324.0  
    27629  74.0  0.0    8.0  265.0  
    27659  74.0  0.0    7.0  206.0  
    27689  74.0  0.0    6.0  206.0  

[102859 rows x 10 columns]

<hr>
<h3> That is it!!! </h3>
<h3> We already have the minimal minimal minimal necessary to run our first ML model! </h3>

So lets do it....
<hr>

In [28]:
# Reset index to get pid and time cols back to the dataframe
X_reseted = X_grped.reset_index()
X_reseted

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum
0,0,29,15.333333,1.5,35.120744,1233.466667,2.445916,5.986385,87.0,0.0,6.0,92.0
1,0,59,13.285714,1.0,32.515198,1057.238095,2.642249,6.985881,87.0,0.0,7.0,93.0
2,0,89,11.625000,1.0,30.467489,928.267857,2.824207,7.982049,87.0,0.0,8.0,93.0
3,0,119,10.333333,1.0,28.761954,827.250000,2.995208,8.978607,87.0,0.0,9.0,93.0
4,0,149,9.300000,0.5,27.313204,746.011111,3.157003,9.975393,87.0,0.0,10.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...
102854,99,27569,32.600000,35.0,28.336078,802.933333,0.034822,-1.788971,74.0,0.0,10.0,326.0
102855,99,27599,36.000000,41.0,27.807373,773.250000,-0.217459,-1.604856,74.0,0.0,9.0,324.0
102856,99,27629,33.125000,35.0,28.261218,798.696429,0.060339,-1.596417,74.0,0.0,8.0,265.0
102857,99,27659,29.428571,29.0,28.359931,804.285714,0.446974,-1.109392,74.0,0.0,7.0,206.0


In [29]:
# Merge X to the original dataframe to append the original activity values
X = pd.merge(X_reseted, df[["time", "pid", "act"]], on=["time", "pid"])
X

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act
0,0,29,15.333333,1.5,35.120744,1233.466667,2.445916,5.986385,87.0,0.0,6.0,92.0,2.0
1,0,59,13.285714,1.0,32.515198,1057.238095,2.642249,6.985881,87.0,0.0,7.0,93.0,0.0
2,0,89,11.625000,1.0,30.467489,928.267857,2.824207,7.982049,87.0,0.0,8.0,93.0,1.0
3,0,119,10.333333,1.0,28.761954,827.250000,2.995208,8.978607,87.0,0.0,9.0,93.0,2.0
4,0,149,9.300000,0.5,27.313204,746.011111,3.157003,9.975393,87.0,0.0,10.0,93.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102854,99,27569,32.600000,35.0,28.336078,802.933333,0.034822,-1.788971,74.0,0.0,10.0,326.0,9.0
102855,99,27599,36.000000,41.0,27.807373,773.250000,-0.217459,-1.604856,74.0,0.0,9.0,324.0,29.0
102856,99,27629,33.125000,35.0,28.261218,798.696429,0.060339,-1.596417,74.0,0.0,8.0,265.0,53.0
102857,99,27659,29.428571,29.0,28.359931,804.285714,0.446974,-1.109392,74.0,0.0,7.0,206.0,74.0


In [30]:
# Get Ys (i.e., sleep labels)
Y = df[["pid", "sleep_phase"]].reset_index().rename(columns={"index": "time"})
Y["sleep"] = Y["sleep_phase"] > 0
Y

,time,pid,sleep_phase,sleep
0,29,0,0.0,False
1,59,0,0.0,False
2,89,0,0.0,False
3,119,0,0.0,False
4,149,0,0.0,False
...,...,...,...,...
102854,3116246,99,0.0,False
102855,3116276,99,0.0,False
102856,3116306,99,0.0,False
102857,3116336,99,0.0,False


### Are there NAs?

In [32]:
X[X.isna().any(axis=1)]

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act
601,0,18059,0.0,0.0,0.000002,3.219017e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
602,0,18089,0.0,0.0,0.000002,3.219017e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
603,0,18119,0.0,0.0,0.000002,3.219017e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
604,0,18149,0.0,0.0,0.000002,3.219017e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
605,0,18179,0.0,0.0,0.000002,3.219017e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102274,99,10169,0.0,0.0,0.000002,5.025142e-12,NaN,NaN,0.0,0.0,11.0,0.0,0.0
102471,99,16079,0.0,0.0,0.000004,1.232615e-11,NaN,NaN,0.0,0.0,11.0,0.0,0.0
102490,99,16649,0.0,0.0,0.000004,1.232629e-11,NaN,NaN,0.0,0.0,11.0,0.0,0.0
102491,99,16679,0.0,0.0,0.000004,1.232629e-11,NaN,NaN,0.0,0.0,11.0,0.0,0.0


Lets have a look at them:

In [33]:
X[590:620]

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act
590,0,17729,50.181818,17.0,73.730344,5.436164e+03,1.703638,1.619478,207.0,0.0,11.0,552.0,1.0
591,0,17759,49.000000,17.0,74.486240,5.548200e+03,1.688307,1.572494,207.0,0.0,11.0,539.0,0.0
592,0,17789,47.454545,5.0,75.387484,5.683273e+03,1.679443,1.531653,207.0,0.0,11.0,522.0,5.0
593,0,17819,28.636364,1.0,54.529392,2.973455e+03,2.616569,7.399291,182.0,0.0,11.0,315.0,48.0
594,0,17849,12.090909,0.0,20.057191,4.022909e+02,1.363299,0.100988,49.0,0.0,11.0,133.0,30.0
595,0,17879,12.090909,0.0,20.057191,4.022909e+02,1.363299,0.100988,49.0,0.0,11.0,133.0,49.0
596,0,17909,12.000000,0.0,20.114671,4.046000e+02,1.362160,0.096118,49.0,0.0,11.0,132.0,0.0
597,0,17939,12.000000,0.0,20.114671,4.046000e+02,1.362160,0.096118,49.0,0.0,11.0,132.0,0.0
598,0,17969,11.545455,0.0,20.343862,4.138727e+02,1.378474,0.106173,49.0,0.0,11.0,127.0,0.0
599,0,17999,7.181818,0.0,16.533712,2.733636e+02,2.208707,4.059921,49.0,0.0,11.0,79.0,0.0


In [34]:
X = X.fillna(0.0)

In [35]:
X[590:620]

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act
590,0,17729,50.181818,17.0,73.730344,5.436164e+03,1.703638,1.619478,207.0,0.0,11.0,552.0,1.0
591,0,17759,49.000000,17.0,74.486240,5.548200e+03,1.688307,1.572494,207.0,0.0,11.0,539.0,0.0
592,0,17789,47.454545,5.0,75.387484,5.683273e+03,1.679443,1.531653,207.0,0.0,11.0,522.0,5.0
593,0,17819,28.636364,1.0,54.529392,2.973455e+03,2.616569,7.399291,182.0,0.0,11.0,315.0,48.0
594,0,17849,12.090909,0.0,20.057191,4.022909e+02,1.363299,0.100988,49.0,0.0,11.0,133.0,30.0
595,0,17879,12.090909,0.0,20.057191,4.022909e+02,1.363299,0.100988,49.0,0.0,11.0,133.0,49.0
596,0,17909,12.000000,0.0,20.114671,4.046000e+02,1.362160,0.096118,49.0,0.0,11.0,132.0,0.0
597,0,17939,12.000000,0.0,20.114671,4.046000e+02,1.362160,0.096118,49.0,0.0,11.0,132.0,0.0
598,0,17969,11.545455,0.0,20.343862,4.138727e+02,1.378474,0.106173,49.0,0.0,11.0,127.0,0.0
599,0,17999,7.181818,0.0,16.533712,2.733636e+02,2.208707,4.059921,49.0,0.0,11.0,79.0,0.0


# First Simple ML model

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score

In [37]:
model = LogisticRegression()
model.fit(X.values[:1000], Y["sleep"].values[:1000])

pred = model.predict(X.values[5000:10000])
f1_score(Y["sleep"].values[5000:10000], pred)

/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7496629213483146

---
# Open Parenthesis
- Is F1 a good metric to use here?
- Is F1 score a good metric in general?

See https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7

In [38]:
f1_score(Y["sleep"].values[5000:10000], pred)

0.7496629213483146

In [39]:
f1_score(~Y["sleep"].values[5000:10000], ~pred)

0.49744360902255647

In [28]:
f1_score?

In [40]:
matthews_corrcoef(~Y["sleep"].values[5000:10000], ~pred), matthews_corrcoef(Y["sleep"].values[5000:10000], pred)

(0.28078064048670404, 0.28078064048670404)

<img src="images/f1.jpg" width="600" height="200">



The usual $f_1$ score is soly based on FP, TP and FN (implicitly coming from Precision and Recall):

$$ F_1 = \frac{2 \times P \times R}{P + R} $$

Matthew's coefficient correlation (Phi correlation)
https://en.wikipedia.org/wiki/Phi_coefficient


$$ MCC = \frac{TP \times TN - FP \times FN}{\sqrt{(TP + FP)\times(TP + FN)\times(TN + FP)\times(TN + FN)}} $$

# Close parenthesis
---

In [44]:
ngrps = 5
pid_grp = {}

i = 0
for pid in X["pid"].unique():
    pid_grp[pid] = i
    i = (i+1) % ngrps

pid_grp

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 0,
 6: 1,
 7: 2,
 8: 3,
 9: 4,
 10: 0,
 11: 1,
 12: 2,
 13: 3,
 14: 4,
 15: 0,
 16: 1,
 17: 2,
 18: 3,
 19: 4,
 20: 0,
 21: 1,
 22: 2,
 23: 3,
 24: 4,
 25: 0,
 26: 1,
 27: 2,
 28: 3,
 29: 4,
 30: 0,
 31: 1,
 32: 2,
 33: 3,
 34: 4,
 35: 0,
 36: 1,
 37: 2,
 38: 3,
 39: 4,
 40: 0,
 41: 1,
 42: 2,
 43: 3,
 44: 4,
 45: 0,
 46: 1,
 47: 2,
 48: 3,
 49: 4,
 50: 0,
 51: 1,
 52: 2,
 53: 3,
 54: 4,
 55: 0,
 56: 1,
 57: 2,
 58: 3,
 59: 4,
 60: 0,
 61: 1,
 62: 2,
 63: 3,
 64: 4,
 65: 0,
 66: 1,
 67: 2,
 68: 3,
 69: 4,
 70: 0,
 71: 1,
 72: 2,
 73: 3,
 74: 4,
 75: 0,
 76: 1,
 77: 2,
 78: 3,
 79: 4,
 80: 0,
 81: 1,
 82: 2,
 83: 3,
 84: 4,
 85: 0,
 86: 1,
 87: 2,
 88: 3,
 89: 4,
 90: 0,
 91: 1,
 92: 2,
 93: 3,
 94: 4,
 95: 0,
 96: 1,
 97: 2,
 98: 3,
 99: 4}

In [45]:
X["grp"] = X["pid"].apply(lambda x: pid_grp[x])
X

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act,grp
0,0,29,15.333333,1.5,35.120744,1233.466667,2.445916,5.986385,87.0,0.0,6.0,92.0,2.0,0
1,0,59,13.285714,1.0,32.515198,1057.238095,2.642249,6.985881,87.0,0.0,7.0,93.0,0.0,0
2,0,89,11.625000,1.0,30.467489,928.267857,2.824207,7.982049,87.0,0.0,8.0,93.0,1.0,0
3,0,119,10.333333,1.0,28.761954,827.250000,2.995208,8.978607,87.0,0.0,9.0,93.0,2.0,0
4,0,149,9.300000,0.5,27.313204,746.011111,3.157003,9.975393,87.0,0.0,10.0,93.0,87.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102854,99,27569,32.600000,35.0,28.336078,802.933333,0.034822,-1.788971,74.0,0.0,10.0,326.0,9.0,4
102855,99,27599,36.000000,41.0,27.807373,773.250000,-0.217459,-1.604856,74.0,0.0,9.0,324.0,29.0,4
102856,99,27629,33.125000,35.0,28.261218,798.696429,0.060339,-1.596417,74.0,0.0,8.0,265.0,53.0,4
102857,99,27659,29.428571,29.0,28.359931,804.285714,0.446974,-1.109392,74.0,0.0,7.0,206.0,74.0,4


In [46]:
scores = cross_val_score(LogisticRegression(), 
                         X.values, 
                         Y["sleep"].values, 
                         groups=X["grp"], 
                         scoring="f1_weighted")
print(scores.mean())
scores

/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

0.8078489342928084


/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.79357922, 0.78114769, 0.81538766, 0.81446029, 0.83466981])

In [47]:
scores = cross_val_score(LogisticRegression(), 
                         X.values, 
                         Y["sleep"].values,
                         groups=X["grp"], 
                         scoring="matthews_corrcoef")
print(scores.mean())
scores

/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

0.5563373079766306


/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.50685162, 0.49896126, 0.57166581, 0.57312517, 0.63108267])

# Ideas to improve??

In [48]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler


pipe = make_pipeline(StandardScaler(), LogisticRegression())
scores = cross_val_score(pipe, X.values, Y["sleep"].values, groups=X["grp"], scoring="matthews_corrcoef")
print(scores.mean())
scores

0.5504742449396713


array([0.49670038, 0.50062669, 0.54590421, 0.57525503, 0.63388492])

# Comparison with different ML techniques -- hand-out test set

In [49]:
test_ids = range(50, 100)

In [50]:
X_test = X[X["pid"].isin(test_ids)]
X_test

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act,grp
51982,50,5099,25.454545,7.0,34.737195,1206.672727,1.189953,-0.005073,94.0,0.0,11.0,280.0,33.0,0
51983,50,5129,28.545455,13.0,33.744225,1138.672727,1.015676,-0.202428,94.0,0.0,11.0,314.0,13.0,0
51984,50,5159,29.090909,13.0,33.281991,1107.690909,1.037608,-0.143160,94.0,0.0,11.0,320.0,7.0,0
51985,50,5189,34.545455,33.0,32.952583,1085.872727,0.640964,-0.888777,94.0,0.0,11.0,380.0,53.0,0
51986,50,5219,36.727273,33.0,31.183620,972.418182,0.660783,-0.684294,94.0,1.0,11.0,404.0,79.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102854,99,27569,32.600000,35.0,28.336078,802.933333,0.034822,-1.788971,74.0,0.0,10.0,326.0,9.0,4
102855,99,27599,36.000000,41.0,27.807373,773.250000,-0.217459,-1.604856,74.0,0.0,9.0,324.0,29.0,4
102856,99,27629,33.125000,35.0,28.261218,798.696429,0.060339,-1.596417,74.0,0.0,8.0,265.0,53.0,4
102857,99,27659,29.428571,29.0,28.359931,804.285714,0.446974,-1.109392,74.0,0.0,7.0,206.0,74.0,4


In [51]:
Y_test = Y[Y["pid"].isin(test_ids)]
Y_test

,time,pid,sleep_phase,sleep
51982,1578416,50,0.0,False
51983,1578446,50,0.0,False
51984,1578476,50,0.0,False
51985,1578506,50,0.0,False
51986,1578536,50,0.0,False
...,...,...,...,...
102854,3116246,99,0.0,False
102855,3116276,99,0.0,False
102856,3116306,99,0.0,False
102857,3116336,99,0.0,False


In [52]:
# The ones that are NOT in the training set are in the test set
X_train = X[~X["pid"].isin(test_ids)]
X_train

,pid,time,mean,median,std,var,skew,kurt,max,min,count,sum,act,grp
0,0,29,15.333333,1.5,35.120744,1233.466667,2.445916,5.986385,87.0,0.0,6.0,92.0,2.0,0
1,0,59,13.285714,1.0,32.515198,1057.238095,2.642249,6.985881,87.0,0.0,7.0,93.0,0.0,0
2,0,89,11.625000,1.0,30.467489,928.267857,2.824207,7.982049,87.0,0.0,8.0,93.0,1.0,0
3,0,119,10.333333,1.0,28.761954,827.250000,2.995208,8.978607,87.0,0.0,9.0,93.0,2.0,0
4,0,149,9.300000,0.5,27.313204,746.011111,3.157003,9.975393,87.0,0.0,10.0,93.0,87.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51977,49,31379,3.454545,0.0,9.832229,96.672727,3.276594,10.802229,33.0,0.0,11.0,38.0,0.0,4
51978,49,31409,4.454545,0.0,10.221190,104.472727,2.656492,7.124992,33.0,0.0,11.0,49.0,0.0,4
51979,49,31439,5.090909,0.0,10.133652,102.690909,2.507661,6.525873,33.0,0.0,11.0,56.0,0.0,4
51980,49,31469,9.727273,0.0,17.539047,307.618182,1.981308,3.310513,53.0,0.0,11.0,107.0,0.0,4


In [53]:
Y_train = Y[~Y["pid"].isin(test_ids)]
Y_train

,time,pid,sleep_phase,sleep
0,29,0,0.0,False
1,59,0,0.0,False
2,89,0,0.0,False
3,119,0,0.0,False
4,149,0,0.0,False
...,...,...,...,...
51977,1573108,49,0.0,False
51978,1573138,49,2.0,True
51979,1573168,49,2.0,True
51980,1573198,49,0.0,False


In [56]:
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe = make_pipeline(LogisticRegression())
pipe.fit(X_train[['mean', 'median', 'std', 'var', 'skew', 'kurt', 'max', 'min', 'count', 'sum', 'act']].values,
         Y_train["sleep"].values)


/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('logisticregression', LogisticRegression())])

In [55]:
y_hat = pipe.predict(X_test[['mean', 'median', 'std', 'var', 'skew', 'kurt', 'max', 'min', 'count', 'sum', 'act']].values)
acc = accuracy_score(Y_test["sleep"].values, y_hat)
f1 = f1_score(Y_test["sleep"].values, y_hat, average="weighted")
mcc = matthews_corrcoef(Y_test["sleep"].values, y_hat)

print("StandardScale + LR: Acc: %.3f, F1 %.3f, MCC: %.3f" % (acc, f1, mcc))

StandardScale + LR: Acc: 0.832, F1 0.816, MCC: 0.571


#### What if we use only the Logistic Regression model?

In [58]:
pipe = LogisticRegression()
pipe.fit(X_train[['mean', 'median', 'std', 'var', 'skew', 'kurt', 'max', 'min', 'count', 'sum', 'act']].values,
         Y_train["sleep"].values)
y_hat = pipe.predict(X_test[['mean', 'median', 'std', 'var', 'skew', 'kurt', 'max', 'min', 'count', 'sum', 'act']].values)
acc = accuracy_score(Y_test["sleep"].values, y_hat)
f1 = f1_score(Y_test["sleep"].values, y_hat, average="weighted")
mcc = matthews_corrcoef(Y_test["sleep"].values, y_hat)
print("Only LR: Acc: %.3f, F1 %.3f, MCC: %.3f" % (acc, f1, mcc))

Only LR: Acc: 0.831, F1 0.815, MCC: 0.568


/Users/joao/miniforge3/envs/tutorial/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
